In [1]:
from sympy import symbols, cos, sin, sign
from sympy.physics.mechanics import dynamicsymbols, ReferenceFrame, Point, RigidBody, outer, KanesMethod
from sympy.physics.mechanics import inertia

## Free-floating Model

Maxwell wheel is modelled as a free floating wheel, where the moment from the tension force is represented as a pure torque on the center of mass.

It kind of fudges the derivation; sympy thinks inertia is defined about zero velocity mass center, but you have to pass it inertia defined about zero velocity point on rope for accurate simulation.

In [2]:
th, uth = dynamicsymbols('theta u_theta') # generalized coordinates and velocities
thdot, uthdot = dynamicsymbols('theta u_theta', 1) # derivatives of gen. coor. and vel.
m, g, r, I = symbols('m g r I') # mass of the wheel

N = ReferenceFrame('N') # Define the reference frame for the wheel
# x is downward, z aligned along wheel's center of rotation
A = N.orientnew('A', 'Axis', [th, N.z]) # wheel body-fixed frame
A.set_ang_vel(N, uth * N.z) # wheel rotates at ang vel. of uth with respect to N.z

MC = Point('MC') # Define mass center of wheel
MC.set_vel(N,0) # Set velocity of mass center

INERTIA = inertia(A,0,0,I) # inertia about the center of mass
W = RigidBody('W', MC, A, m, (INERTIA, MC)) # Rigid body representing wheel

kd = [thdot - uth] # kinematic differential equation

# FL = [(A,-m*g*r*sign(th)*A.z)] # force list
FL = [(A, -m*g*r*sign(th)*A.z)] # moment changes direction depending on sign of angular displacement
BL = [W] # body list

In [3]:
KM = KanesMethod(N, q_ind=[th], u_ind=[uth], kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BL, FL)

In [4]:
# KM.rhs()
MASS = KM.mass_matrix
FORCING = KM.forcing
print(MASS)
print(FORCING)
# rhs = MASS.inv() * FORCING

Matrix([[I]])
Matrix([[-g*m*r*sign(theta(t))]])


## Rolling Model

In [2]:
# Assume motion of the Maxwell wheel can be modelled as pure rotation about the rope

th, uth = dynamicsymbols('theta u_theta') # generalized coordinates and velocities
thdot, uthdot = dynamicsymbols('theta u_theta', 1) # derivatives of gen. coor. and vel.
m, g, r, I = symbols('m g r I') # mass of the wheel

N = ReferenceFrame('N') # Define the reference frame for the wheel
# x is downward, y is to the right, z aligned along wheel's center of rotation
A = N.orientnew('A', 'Axis', [th, N.z]) # wheel body-fixed frame
A.set_ang_vel(N, uth * N.z) # wheel rotates at ang vel. of uth with respect to N.z

IC = Point('IC') # Define zero velocity point on wheel
IC.set_vel(N,0) # Set velocity of IC point
MC = IC.locatenew('MC', r*sign(th)*N.y) # Point of wheel center of mass
# MC.v2pt_theory(IC, N, A) # velocity of center of mass
MC.set_vel(N,-r*uth*sign(th)*N.x)

# I = outer(A.z, A.z) # Dyadic for inertia about center of mass
INERTIA = inertia(A,0,0,I) # inertia about the center of mass
W = RigidBody('W', MC, A, m, (INERTIA, MC)) # Rigid body representing wheel

kd = [thdot - uth] # kinematic differential equation

# FL = [(A,-m*g*r*sign(th)*A.z)] # force list
FL = [(MC, m*g*N.x)]
BL = [W] # body list

In [3]:
KM = KanesMethod(N, q_ind=[th], u_ind=[uth], kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BL, FL)

In [4]:
# KM.rhs()
MASS = KM.mass_matrix
FORCING = KM.forcing
print(MASS)
print(FORCING)
# rhs = MASS.inv() * FORCING

Matrix([[I + m*r**2*sign(theta(t))**2]])
Matrix([[-g*m*r*sign(theta(t)) - m*r**2*u_theta(t)*sign(theta(t))*Derivative(sign(theta(t)), t)]])
